## quest
- train_test_split() 적용
- train과 test set 대한 classification_report() 적용

## 지도학습(Supervised Learning)

In [49]:
import pandas as pd

In [50]:
df_BCD = pd.read_csv('../../../datasets/BreastCancerWisconsinDataSet.csv')
df_BCD[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [51]:
df_BCD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [52]:
df_BCD.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

### 목표변수 - 범주형
- 목표변수 : diagnosis
- 설명변수 : 'radius_se', 'area_se', 'area_worst'

### 전처리 과정

In [53]:
df_BCD_extract = df_BCD[['diagnosis', 'radius_se', 'area_se', 'area_worst']]
df_BCD_extract.isnull().sum()

diagnosis     0
radius_se     0
area_se       0
area_worst    0
dtype: int64

#### diagnosis의 values  전처리 
- 머신러닝을 하기 위해서는 숫자로만 이루어져야함.
- B, M을 0,1과 전환 

In [54]:
df_BCD_extract['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [55]:
def convert_diagnosis(diagnosis):
    if diagnosis == 'B':
        return 0
    else :
        return 1

In [56]:
df_BCD_extract['diagnosis_preprocessing'] = df_BCD_extract['diagnosis'].apply(convert_diagnosis)
df_BCD_extract[:2]

C:\Users\02-19\AppData\Local\Temp\ipykernel_11744\78250299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BCD_extract['diagnosis_preprocessing'] = df_BCD_extract['diagnosis'].apply(convert_diagnosis)


,diagnosis,radius_se,area_se,area_worst,diagnosis_preprocessing
0,M,1.0950,153.40,2019.0,1
1,M,0.5435,74.08,1956.0,1


### 정형화단계

In [57]:
target_diagnosis = df_BCD_extract['diagnosis_preprocessing']
features_diagnosis = df_BCD_extract.drop(columns=['diagnosis_preprocessing','diagnosis'])
target_diagnosis.shape, features_diagnosis.shape

((569,), (569, 3))

#### split 

In [58]:
from sklearn.model_selection import train_test_split
target = df_BCD_extract['diagnosis_preprocessing']
features = df_BCD_extract.drop(columns=['diagnosis_preprocessing','diagnosis'])

In [59]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111) #random_state=111 랜덤값으로 설정
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((426, 3), (426,), (143, 3), (143,))

### 모델 학습

In [60]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() ## 인스턴스화 
model.fit(features_diagnosis, target_diagnosis)

LogisticRegression()

In [61]:
model.coef_, model.intercept_

(array([[0.31482089, 0.03269899, 0.01053156]]), array([-10.08174371]))

### 예측

In [62]:
df_BCD_extract[100:105]

,diagnosis,radius_se,area_se,area_worst,diagnosis_preprocessing
100,M,0.4565,43.140,906.5,1
101,B,0.2241,9.833,185.2,0
102,B,0.1924,14.680,547.8,0
103,B,0.1803,11.770,361.2,0
104,B,0.3550,23.130,402.8,0


In [63]:
model.predict(features_diagnosis[100:105])

## 실제값 : 1,0,0,0,0 / 예측값 : 1, 0, 0, 0, 0]

array([1, 0, 0, 0, 0], dtype=int64)

In [64]:
model.predict_proba(features_diagnosis[100:105])

## 0일 확률 :2.65131056e-01 / 1일 확률 : 7.34868944e-01

array([[2.65131056e-01, 7.34868944e-01],
       [9.99564777e-01, 4.35222524e-04],
       [9.77510511e-01, 2.24894890e-02],
       [9.97088334e-01, 2.91166589e-03],
       [9.93115273e-01, 6.88472701e-03]])

### 평가 

In [65]:
target_train_predict = model.predict(features_train)
target_train_predict.shape

(426,)

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
accuracy_score(target_train, target_train_predict) 
## 교육한값과 교육을 예측한 값과 비교(교내 시험)

0.9154929577464789

In [68]:
target_test_predict = model.predict(features_test) # test한 값의 예측값
target_test_predict.shape #target_test.shape과 동일

(143,)

In [69]:
accuracy_score(target_test, target_test_predict)
## 실제test값과 시험값 예측의 비교(교외 시험)

0.9090909090909091

#### F1
- shape에서 갯수가 같아야 report를 쓸수 있음 

In [70]:
from sklearn.metrics import classification_report

In [71]:
print(classification_report(target_train, target_train_predict)) #교육

## support ; record의 갯수 
## accuracy : 0.92 / f1-score : 0.94

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       271
           1       0.93      0.83      0.88       155

    accuracy                           0.92       426
   macro avg       0.92      0.90      0.91       426
weighted avg       0.92      0.92      0.91       426



In [72]:
print(classification_report(target_test, target_test_predict)) #시험

## accuracy : 0.91 /f1-score :0.92 
# 값의 치우침이 없어보임

              precision    recall  f1-score   support

           0       0.92      0.93      0.92        86
           1       0.89      0.88      0.88        57

    accuracy                           0.91       143
   macro avg       0.91      0.90      0.90       143
weighted avg       0.91      0.91      0.91       143



### 서비스 배포

In [73]:
import pickle

In [74]:
with open('../../../datasets/BreastCancerWisconsinDataSet.pkl','wb') as regression_file : 
    pickle.dump(obj=model, file=regression_file)
    pass